## Test aws connectivity
To test the connectivity to AWS using Python and Boto3, you can try to list the available regions in your account. Here's an example Python script:

In [ ]:
import boto3

# Create a Boto3 client for the Amazon EC2 service
ec2_client = boto3.client('ec2')

# List the available regions
regions = ec2_client.describe_regions()
for region in regions['Regions']:
    print(region['RegionName'])

# Ask for user confirmation to continue
user_input = input('Do you want to continue with the script? (Y/N): ').lower()
if user_input != 'y':
    print('Exiting the script...')
    exit()

# Continue with the rest of the script...


In this script, we create a Boto3 client for the Amazon EC2 service without specifying a region, which means that Boto3 will use the default region configured in your AWS CLI or environment variables. We then call the describe_regions method to list the available regions and print their names.

You can run this script in a Python environment that has Boto3 installed and configured with valid AWS credentials. If the script runs without errors and lists the available regions, then it indicates that your connectivity to AWS is working. If you encounter errors, it may be due to invalid or expired credentials, misconfigured AWS CLI or environment variables, or network connectivity issues.

### python script to list aws fargate instances
To list AWS Fargate instances using Python, you will need to use the AWS SDK for Python (Boto3). Here's an example script that lists all Fargate instances in a given AWS region:

import boto3

# Specify the AWS region where your Fargate instances are running
region_name = 'us-west-2'

# Create a Boto3 client for the Amazon ECS service
ecs_client = boto3.client('ecs', region_name=region_name)

# List the Fargate instances
response = ecs_client.list_container_instances(
    cluster='default',
    status='ACTIVE',
    launchType='FARGATE'
)

# Print the ARN of each Fargate instance
for arn in response['containerInstanceArns']:
    print(arn)


In this example, we create a Boto3 client for the Amazon ECS service, which provides APIs to manage Fargate instances. We then call the list_container_instances method to list all active Fargate instances in the default ECS cluster. We filter the instances by their launch type, which must be FARGATE. Finally, we print the ARN of each Fargate instance returned by the API. You can modify the script to filter instances by other attributes, such as their status or tags.

## Create a list of the fargate instances to the user for selection to restart
To provide a list of Fargate instances to the user for selection and restart, you can modify the previous Python script to add an interactive menu that displays the instance ARNs and prompts the user to select one or more instances to restart. Here's an example:

In [ ]:
import boto3

# Specify the AWS region where your Fargate instances are running
region_name = 'us-west-2'

# Create a Boto3 client for the Amazon ECS service
ecs_client = boto3.client('ecs', region_name=region_name)

# List the Fargate instances
response = ecs_client.list_container_instances(
    cluster='default',
    status='ACTIVE',
    launchType='FARGATE'
)

# Build a dictionary that maps instance ARNs to their IDs
instance_ids = {}
for arn in response['containerInstanceArns']:
    instance_id = ecs_client.describe_container_instances(
        cluster='default',
        containerInstances=[arn]
    )['containerInstances'][0]['ec2InstanceId']
    instance_ids[arn] = instance_id

# Print a menu of the Fargate instances and prompt the user to select one or more
print('Select one or more Fargate instances to restart:')
for i, arn in enumerate(instance_ids):
    print(f"{i+1}. {arn} (Instance ID: {instance_ids[arn]})")
selections = input('Enter one or more numbers (comma-separated): ').split(',')
selected_instances = [list(instance_ids.keys())[int(i)-1] for i in selections]

# Restart the selected Fargate instances
for arn in selected_instances:
    ecs_client.update_container_instances_state(
        cluster='default',
        containerInstances=[arn],
        status='DRAINING'
    )


In this example, we first build a dictionary that maps instance ARNs to their corresponding EC2 instance IDs. We then print a menu of the Fargate instances, displaying their ARNs and instance IDs, and prompt the user to select one or more instances to restart by entering the corresponding numbers separated by commas. We then restart the selected instances by calling the update_container_instances_state method with a status of DRAINING, which causes the instances to be removed from service and replaced with new instances. You can modify the script to perform other operations on the selected instances, such as updating their configurations or terminating them.

## Add Monitoring for Active Status

To monitor the restarted Fargate instances for availability, you can use the AWS SDK for Python (Boto3) to check the status of the instances and wait until they are back to the ACTIVE state. Here's an example script that restarts the selected Fargate instances and waits for them to become available:

In [ ]:
import boto3
import time

# Specify the AWS region where your Fargate instances are running
region_name = 'us-west-2'

# Create a Boto3 client for the Amazon ECS service
ecs_client = boto3.client('ecs', region_name=region_name)

# List the Fargate instances
response = ecs_client.list_container_instances(
    cluster='default',
    status='ACTIVE',
    launchType='FARGATE'
)

# Build a dictionary that maps instance ARNs to their IDs
instance_ids = {}
for arn in response['containerInstanceArns']:
    instance_id = ecs_client.describe_container_instances(
        cluster='default',
        containerInstances=[arn]
    )['containerInstances'][0]['ec2InstanceId']
    instance_ids[arn] = instance_id

# Print a menu of the Fargate instances and prompt the user to select one or more
print('Select one or more Fargate instances to restart:')
for i, arn in enumerate(instance_ids):
    print(f"{i+1}. {arn} (Instance ID: {instance_ids[arn]})")
selections = input('Enter one or more numbers (comma-separated): ').split(',')
selected_instances = [list(instance_ids.keys())[int(i)-1] for i in selections]

# Restart the selected Fargate instances
for arn in selected_instances:
    ecs_client.update_container_instances_state(
        cluster='default',
        containerInstances=[arn],
        status='DRAINING'
    )

# Wait for the instances to become available again
while True:
    response = ecs_client.list_container_instances(
        cluster='default',
        containerInstanceIds=list(instance_ids.values())
    )
    statuses = [instance['status'] for instance in response['containerInstances']]
    if all(status == 'ACTIVE' for status in statuses):
        print('All instances are back to ACTIVE state')
        break
    else:
        print(f"Some instances are still {statuses.count('DRAINING')} in DRAINING state...")
        time.sleep(10)


In this example, after restarting the selected Fargate instances, we use a while loop to repeatedly call the list_container_instances method to check the status of the instances. We store the statuses in a list and check if all of them are ACTIVE. If so, we print a message indicating that all instances are back to the ACTIVE state and break out of the loop. Otherwise, we print a message indicating how many instances are still in the DRAINING state and wait for 10 seconds before checking again. You can modify the script to perform other checks on the instances, such as verifying that they are serving requests correctly.

### Adding in ser input for configuration and cluster to use
Here's a Python script that tests AWS connectivity, lists Fargate instances, prompts the user for which instance to restart, and tests until the instance has been restarted:

In [ ]:
import subprocess
import sys

# Check for and install prerequisites
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import boto3
    print("boto3 is installed.")
except ImportError:
    print("boto3 is not installed. Installing now...")
    install('boto3')
    import boto3

try:
    subprocess.check_call(['aws', '--version'])
    print("AWS CLI is installed.")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("AWS CLI is not installed. Installing now...")
    install('awscli')

# Ask for AWS configuration
user_input = input("Enter the number of the AWS configuration to use: \n1. Default\n2. Custom\n")
if user_input == "1":
    # Use default configuration
    ec2 = boto3.client('ec2')
    ecs = boto3.client('ecs')
elif user_input == "2":
    # Use custom configuration
    aws_access_key_id = input("Enter the AWS access key ID: ")
    aws_secret_access_key = input("Enter the AWS secret access key: ")
    region_name = input("Enter the region name: ")
    ec2 = boto3.client(
        'ec2',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name
    )
    ecs = boto3.client(
        'ecs',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name
    )
else:
    print("Invalid input.")
    exit()

# Test AWS connectivity
try:
    response = ec2.describe_instances()
    print("AWS connectivity is working.")
except ClientError as e:
    print(e)
    exit()

# Ask to continue
user_input = input("Do you want to continue? (y/n): ")
if user_input.lower() != "y":
    exit()

# List clusters
try:
    response = ecs.list_clusters()
    clusters = response['clusterArns']
    if len(clusters) == 0:
        print("No ECS clusters found.")
        exit()
    else:
        print("List of ECS clusters:")
        for i, cluster in enumerate(clusters):
            print(f"{i+1}. {cluster}")
except ClientError as e:
    print(e)
    exit()

# Prompt for cluster
user_input = input("Enter the number of the ECS cluster: ")
try:
    cluster_number = int(user_input) - 1
    if cluster_number < 0 or cluster_number >= len(clusters):
        print("Invalid input.")
        exit()
    cluster = clusters[cluster_number]
except ValueError:
    print("Invalid input.")
    exit()

# List Fargate instances in the selected cluster
try:
    response = ecs.list_tasks(
        cluster=cluster,
        launchType='FARGATE',
        desiredStatus='RUNNING'
    )
    tasks = response['taskArns']
    if len(tasks) == 0:
        print("No Fargate instances found.")
        exit()
    else:
        print("List of Fargate instances:")
        for i, task in enumerate(tasks):
            response = ecs.describe_tasks(
                cluster=cluster,
                tasks=[task]
            )
            task_definition = response['tasks'][0]['taskDefinitionArn']
            print(f"{i+1}. {task_definition}")
except ClientError as e:
    print(e)
    exit()

# Prompt for instance to restart
user_input = input("Enter the number of the Fargate instance to restart: ")
try:
    instance_number = int(user_input) - 1
    if instance_number < 0 or instance_number >= len(tasks):
        print("Invalid input.")
        exit()
    instance = tasks[instance_number]
except ValueError:
    print("Invalid input.")
    exit()

# Restart instance
try:
    response = ecs.stop_task(
        cluster=cluster,
        task=instance,
        reason='Restarting task'
    )
    print("Task stopped.")
except ClientError as e:
    print(e)
    exit()

while True:
    # Check if instance is running
    response = ecs.describe_tasks(
        cluster=cluster,
        tasks=[instance]
    )
    task_status = response['tasks'][0]['lastStatus']
    if task_status == 'STOPPED':
        print("Task has been restarted.")
        break
    print("Task is still stopping...")
    time.sleep(5)

Notes:

    * The list_clusters function is used to list the ECS clusters. The user is prompted to choose a cluster by entering the corresponding number.
    * The cluster parameter is added to the list_tasks, describe_tasks, and stop_task functions to specify the cluster where the Fargate instances are located.
    * The stop_task function is used to restart the Fargate instance. This function stops the task and starts a new one with the same task definition and other settings.



### Programmatically defined configuration

In [ ]:
import subprocess
import sys
import boto3

# Check for and install prerequisites
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import boto3
    print("boto3 is installed.")
except ImportError:
    print("boto3 is not installed. Installing now...")
    install('boto3')
    import boto3

try:
    subprocess.check_call(['aws', '--version'])
    print("AWS CLI is installed.")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("AWS CLI is not installed. Installing now...")
    install('awscli')

# Define AWS configuration
def get_aws_config():
    return boto3.session.Session().get_credentials()

aws_config = get_aws_config()

ec2 = boto3.client(
    'ec2',
    aws_access_key_id=aws_config.access_key,
    aws_secret_access_key=aws_config.secret_key,
    aws_session_token=aws_config.token,
    region_name='us-east-1'
)

ecs = boto3.client(
    'ecs',
    aws_access_key_id=aws_config.access_key,
    aws_secret_access_key=aws_config.secret_key,
    aws_session_token=aws_config.token,
    region_name='us-east-1'
)

# Test AWS connectivity
try:
    response = ec2.describe_instances()
    print("AWS connectivity is working.")
except Exception as e:
    print(e)
    exit()

# Rest of the script...
